In [1]:
import os

In [2]:
%pwd

'D:\\CS\\iNeuron\\Deep Learning\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'D:\\CS\\iNeuron\\Deep Learning\\Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.utils import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "chicken-fecal-images")

        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import time
import tensorflow as tf


class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )

    @property
    def _create_early_stopping(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )

    @property
    def _create_lr_scheduler(self):
        def scheduler(epoch, lr):
            if epoch % 10 == 0 and epoch > 0:
                lr = lr / 2
            return lr

        return tf.keras.callbacks.LearningRateScheduler(scheduler)

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
            self._create_early_stopping,
            self._create_lr_scheduler
        ]

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.validation_steps = None
        self.steps_per_epoch = None
        self.train_generator = None
        self.valid_generator = None
        self.model = None
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        data_generator_kwargs = dict(
            rescale=1. / 255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bi-linear"
        )

        valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_generator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **data_generator_kwargs
            )
        else:
            train_data_generator = valid_generator

        self.train_generator = train_data_generator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
except Exception as e:
    raise e

[2023-07-22 11:37:37,573: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-07-22 11:37:37,582: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-07-22 11:37:37,585: INFO: utils: created directory at: artifacts]
[2023-07-22 11:37:37,587: INFO: utils: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-07-22 11:37:37,590: INFO: utils: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-07-22 11:37:37,593: INFO: utils: created directory at: artifacts\training]
Found 831 images belonging to 2 classes.
Found 3329 images belonging to 2 classes.
Epoch 1/50
104/104 [==============================] - 1237s 12s/step - loss: 5.3090 - accuracy: 0.7225 - val_loss: 3.9976 - val_accuracy: 0.6737
Epoch 2/50


C:\Users\tejan\anaconda3\envs\chicken\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


104/104 [==============================] - 1286s 12s/step - loss: 1.2753 - accuracy: 0.8826 - val_loss: 0.2739 - val_accuracy: 0.9550
Epoch 3/50
104/104 [==============================] - 1251s 12s/step - loss: 0.3757 - accuracy: 0.9381 - val_loss: 0.2236 - val_accuracy: 0.9600
Epoch 4/50
104/104 [==============================] - 1261s 12s/step - loss: 0.3192 - accuracy: 0.9418 - val_loss: 0.1464 - val_accuracy: 0.9700
Epoch 5/50
104/104 [==============================] - 1677s 16s/step - loss: 0.5461 - accuracy: 0.9296 - val_loss: 0.1354 - val_accuracy: 0.9725
Epoch 6/50
104/104 [==============================] - 1842s 18s/step - loss: 1.0054 - accuracy: 0.9445 - val_loss: 0.1467 - val_accuracy: 0.9688
Epoch 7/50
104/104 [==============================] - 2009s 19s/step - loss: 0.2934 - accuracy: 0.9451 - val_loss: 0.1326 - val_accuracy: 0.9750
Epoch 8/50
104/104 [==============================] - 1337s 13s/step - loss: 0.2534 - accuracy: 0.9484 - val_loss: 0.2780 - val_accuracy: 0.9

KeyboardInterrupt: 